# 🏎️ F1 Pre-Race Excitement Forecaster

## 💭 Predictive Task

The goal of our project is to build a binary classification model that predicts whether a Formula 1 race will be **“Exciting”** (1) or **“Not Exciting”** (0) before the race begins.
All predictions must rely solely on pre-race features, such as:

- Qualifying results and grid spread
- Championship standings (point gaps, title contention)
- Circuit characteristics (altitude, layout properties)

The ground-truth labels are generated from post-race data using objective thresholds on:
- Number of DNFs
- Total overtakes

A race is labeled Exciting if it falls in the top 25% in either category.

## Background

Formula 1 is widely regarded as the pinnacle of motorsport, yet the excitement level of each Grand Prix can vary dramatically. Some races unfold as predictable processions, while others deliver unexpected chaos, dramatic overtakes, and major shifts in championship momentum. This inconsistency makes it difficult for casual viewers to decide which races are worth watching, and even dedicated fans often gravitate toward events that promise high action. Predicting that excitement before a race begins, however, is far from straightforward. The dynamics that shape a race’s entertainment value — grid order, competitive pressure, track layout, and environmental conditions — all interact in complex ways that are not immediately obvious.

In recent years, the narrative surrounding a race before it happens has become almost as influential as the race itself. Tight championship battles, surprising qualifying results, unusual track environments, or major penalties can all heighten expectations for an unpredictable event. Although excitement is inherently subjective, many measurable factors correlate with how action-packed a race turns out to be. Metrics such as overtakes, retirements, and field spread provide concrete signals of how dynamic a race was. These relationships motivate our central goal: to build a machine learning classifier capable of predicting a race’s entertainment value *strictly* from ***pre-race information***.

To explore this, we draw on historical Formula 1 data spanning 1950–2024, focusing on features that can be known before lights out — including qualifying performance, driver standings, and circuit characteristics. We then generate an objective “excitement” label using post-race metrics such as DNFs and total overtakes. This allows us to train and evaluate our model without introducing future information into the predictions. By combining pre-race context with well-defined labels, our project investigates whether it is possible to anticipate the entertainment potential of a Grand Prix ahead of time. Such a model can help casual fans choose which races to watch live and may provide broader insight into how pre-race conditions shape the dynamics of Formula 1 competition.

## 🔍 Prior Work

Research on predicting the entertainment value of sporting events has grown a lot in recent years, especially in football, basketball, and various motorsports. Most of that work, however, focuses on performance outcomes — things like predicting the race winner, estimating lap times, or modeling podium finishes. In Formula 1 specifically, there’s been work using historical timing data to estimate pace differences, applying machine learning to forecast pit strategies, or studying how track layout affects overtaking. These kinds of projects show that many aspects of a race can be anticipated using pre-race information.

For Formula 1, several Kaggle notebooks and independent analyses have also looked at how overtakes relate to circuit characteristics, how altitude or temperature impacts power units, and how qualifying gaps influence race-day competitiveness. A lot of these takeaways point toward similar themes: grid variability, championship pressure, and track features all play meaningful roles in shaping how a race plays out — which aligns with the features we build into our model.

What’s much less common is trying to classify the overall “excitement” of a race as its own prediction task. The few attempts that exist usually rely on fan surveys or subjective scoring systems, which can be inconsistent and hard to scale. 

Our approach differs in two important ways:

**Objective Labeling**:
- Instead of using opinions or ratings, we define excitement using measurable post-race metrics like DNFs and overtakes. This gives us a consistent, reproducible label.

**Strictly Pre-Race Features**:
- We only use information available before the race starts, which helps avoid data leakage — something not all prior work accounts for.

By combining insights from motorsport analytics with this stricter setup, our project pushes into a relatively under-explored space: trying to predict how entertaining a Formula 1 race will be before anyone has turned a wheel.

## 📊 Data

#### Dataset #1 — Formula 1 World Championship Dataset

Dataset Name: [Formula 1 World Championship (1950–2024)](https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020)

Number of observations: ~80,000+ records across all tables

Number of variables: Varies by table (5–20 columns each)

This dataset contains the official historical results from every Formula 1 race, including tables such as races, results, qualifying, driver_standings, circuits, and lap_times. Important variables include race metadata (year, round, circuit), driver finishing positions, qualifying times, lap-by-lap positions, and driver point standings. These variables represent concrete performance and contextual indicators such as competitiveness, grid spread, pace variability, and circuit characteristics. For preprocessing, we filtered the data to modern-era races (year ≥ 2000), handled null values, merged weather data, and removed irrelevant scheduling columns before feature engineering and label creation.

#### Dataset #2 — Formula 1 Weather Dataset

Dataset Name: [Formula 1 Race Weather Info (1950–2024)](https://www.kaggle.com/datasets/mariyakostyrya/formula-1-weather-info-1950-2024)

Number of observations: ~1,500

Number of variables: 10

This dataset provides race-day weather conditions such as temperature, precipitation, humidity, and wind speed for each Grand Prix. Key variables we use include temperature (float), precipitation (float), and a derived feature is_rain, which captures whether the track was wet. Weather serves as a proxy for external race conditions that can influence excitement, risk-taking, and chaos (e.g., wet races tend to produce more DNFs and overtakes). Preprocessing steps included converting timestamps into year/round identifiers, filling missing values with reasonable defaults, and merging the data into the main race table.

#### Dataset #3 — Combined Race-Level Dataset (Engineered)

Dataset Name: F1 Race-Level Modeling Dataset (Engineered)

Link: Locally created from dataset #1 and #2

Number of observations:1000–1100 races (depending on filtering)

Number of variables: ~20 engineered features

This final dataset is constructed by merging the cleaned race metadata with pre-race features (qualifying spread, grid shake-ups, championship tension) and the weather variables. Additional post-race metrics from results and lap_times are used only to create the ground-truth excitement labels (DNFs → chaos score, position changes → action score). Preprocessing involved merging multiple CSVs on raceId, calculating new features, filtering to modern seasons, and ensuring no data leakage by keeping all post-race information strictly for label generation.

In [ ]:
# --- Imports & Configuration ---
import pandas as pd
import numpy as np
import io
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration
DATA_PATH = 'data/'  # Folder where you unzipped the Kaggle dataset
START_YEAR = 2000    # We focus on the modern era for data consistency

In [ ]:
# --- Data Loading ---
def load_data():
    print("Loading Vopani Dataset CSVs...")
    # Load core files with '\N' handling for nulls
    races = pd.read_csv(f'{DATA_PATH}races.csv', na_values='\\N')
    results = pd.read_csv(f'{DATA_PATH}results.csv', na_values='\\N')
    status = pd.read_csv(f'{DATA_PATH}status.csv', na_values='\\N')
    qualifying = pd.read_csv(f'{DATA_PATH}qualifying.csv', na_values='\\N')
    standings = pd.read_csv(f'{DATA_PATH}driver_standings.csv', na_values='\\N')
    circuits = pd.read_csv(f'{DATA_PATH}circuits.csv', na_values='\\N')
    lap_times = pd.read_csv(f'{DATA_PATH}lap_times.csv', na_values='\\N')
    weather = pd.read_csv(f'{DATA_PATH}weather.csv', na_values='\\N')
    print("Datasets are loaded in...")

    return races, results, status, qualifying, standings, circuits, lap_times, weather

# Load the raw dataframes
races_df, results_df, status_df, qual_df, stand_df, circuits_df, laps_df, weather_df = load_data()

In [ ]:
# --- Cleaning & Weather Merging ---
def clean_and_merge_weather(races, weather):
    print("Processing & Merging Weather Data...")
    
    # 1. Extract Year from Weather Date
    weather['datetime'] = pd.to_datetime(weather['datetime'])
    weather['year'] = weather['datetime'].dt.year

    # 3. Merge Weather with Official Race List
    # We join on ['year', 'round'] to align the weather with the correct Race ID
    merged_races = pd.merge(
        races, 
        weather[['year', 'round', 'temperature', 'precipitation']], 
        on=['year', 'round'], 
        how='left'
    )
    
    # 4. Filter for Modern Era (2000+)
    # merged_races = merged_races[merged_races['year'] >= START_YEAR].copy()
    
    # 5. Handle Missing Weather (Missing)
    # We fill with defaults (Dry, 20°C) so the model doesn't crash
    merged_races['precipitation'] = merged_races['precipitation'].fillna(0)
    merged_races['temperature'] = merged_races['temperature'].fillna(20.0)

    drop_cols = [
        'url', 
        'fp1_date', 'fp1_time', 
        'fp2_date', 'fp2_time', 
        'fp3_date', 'fp3_time', 
        'quali_date', 'quali_time', 
        'sprint_date', 'sprint_time'
    ]
    
    # Only drop columns that actually exist in the dataframe to avoid errors
    cols_to_drop = [c for c in drop_cols if c in merged_races.columns]
    
    df_clean = merged_races.drop(columns=cols_to_drop)
    
    print(f"Total Races processed: {merged_races.shape[0]}")
    return df_clean

# Execute Cleaning
main_races = clean_and_merge_weather(races_df, weather_df)
# Apply the cleaning

# Verify what's left
print("Columns kept:", main_races.columns.tolist())
main_races.head()

In [ ]:
# --- Generating Labels (Y) ---
def generate_labels(races, results, status, laps):
    print("Calculating Excitement Labels...")
    
    # counts of unique race id
    target_race_ids = races['raceId'].unique()
    
    # --- A. Calculate CHAOS Score (DNFs) ---
    # Chaos is a metric determined by how drivers finished a race out of the participating drivers
    # The drivers that finished a race, the more likely that the chaos score is lowered

    # Copy from results df to get filtered unique race ids
    results_filtered = results[results['raceId'].isin(target_race_ids)].copy()
    
    # Status IDs 1, 11-19 represent normal finishes. Everything else is a DNF.
    # In status.csv, ids 1 is considered a normal finish, ids 11-19 refer to how many laps the leader lapped this driver
    # The # of laps that the driver was lapped is id - 10 (e.g 11-10 = +1 laps)
    # We will consider all types of finishes as a safe finish
    normal_finish_ids = [1] + list(range(11, 20))

    # Create a new dataframe column name 'is_dnf' 
    # Checks if drivers in results_filtered had a normal finish
    # If true, the tilde inverts (NOT operator) to false and vice versa
    results_filtered['is_dnf'] = ~results_filtered['statusId'].isin(normal_finish_ids)
    
    # Group results_filtered by the race and sum everything in the 'is_dnf' column
    # Rename the column of the resulting dataframe into a chaos score
    chaos_df = results_filtered.groupby('raceId')['is_dnf'].sum().reset_index()
    chaos_df.rename(columns={'is_dnf': 'chaos_score'}, inplace=True)
    
    # --- B. Calculate ACTION Score (Overtakes/Position Changes) ---
    # Filter laps from laptimes.csv with each unique race ids.
    laps_filtered = laps[laps['raceId'].isin(target_race_ids)].copy()
    laps_filtered.sort_values(['raceId', 'driverId', 'lap'], inplace=True)
    
    # Calculate position change from previous lap
    # Sort by race, driver, and position
    # This line takes a unique driver's position in between laps
    # diff() computes the difference of current row to previous row
    laps_filtered['pos_change'] = laps_filtered.groupby(['raceId', 'driverId'])['position'].diff().abs()

    # Group results by new 'pos_change' column to include only the race and position and sum on the position values
    # High values mean more position changes = suggests more action
    # Low values mean steady position placement = suggest "boring" race
    action_df = laps_filtered.groupby('raceId')['pos_change'].sum().reset_index()
    action_df.rename(columns={'pos_change': 'action_score'}, inplace=True)
    
    # --- C. Create Final Label ---
    # Merge the previous metrics given each unique race
    metrics = races[['raceId']].merge(chaos_df, on='raceId', how='left')
    metrics = metrics.merge(action_df, on='raceId', how='left').fillna(0)
    
    # Define "Exciting" as Top 25% in Chaos OR Top 25% in Action
    chaos_thresh = metrics['chaos_score'].quantile(0.85)
    action_thresh = metrics['action_score'].quantile(0.85)
    
    metrics['is_exciting'] = (
        (metrics['chaos_score'] > chaos_thresh) | 
        (metrics['action_score'] > action_thresh)
    ).astype(int)
    
    print(f"Thresholds -> Chaos: >{chaos_thresh} DNFs, Action: >{action_thresh} Changes")
    return metrics[['raceId', 'is_exciting']]

# Execute Label Generation
labels_df = generate_labels(main_races, results_df, status_df, laps_df)
print(labels_df)

## EDA

In [ ]:
def convert_laptime(x):
    if pd.isna(x):
        return None
    if isinstance(x, (int, float)):  # already numeric
        return x
    m, s = x.split(':')
    return float(m)*60 + float(s)

def clean_laptime(t):
    if pd.isna(t):
        return np.nan
    if t < 30 or t > 600:   # unrealistic for any F1 circuit in any era
        return np.nan
    return t

for col in ['q1','q2','q3']:
    qual_df[col] = qual_df[col].apply(convert_laptime)
    qual_df[col] = qual_df[col].apply(clean_laptime)

Merge qualifying lap times with year to allow for the computation of future features

In [ ]:
qual_df = qual_df.merge(
    races_df[['raceId', 'year', 'round']],
    on='raceId',
    how='left'
)

qual_df['best_time'] = qual_df[['q3', 'q2', 'q1']].bfill(axis=1).iloc[:, 0]

#### Grid Spread Time
Grid Spread Time represents the amount of variation in drivers’ qualifying lap times before a race. It is calculated as the standard deviation of all qualifying times for a given event.

In other words, it measures how tightly grouped or spread out the drivers were in terms of raw pace during qualifying:

- Low Grid Spread Time → drivers posted very similar lap times, meaning the field was tightly matched.
- High Grid Spread Time → lap times varied widely, indicating a larger performance gap between drivers.

This feature is relevant because qualifying performance often influences how competitive a race will be. A tightly grouped field may lead to closer battles during the race, while a widely spread field may result in fewer on-track position changes.

However, as shown in our KDE analysis, the distribution of Grid Spread Time is very similar for both exciting and non-exciting races. This suggests that Grid Spread Time alone does not strongly differentiate race excitement levels and provides limited predictive power for classification.

In [ ]:
# ---------- GRID SPREAD ----------
grid_spread_time = (
    qual_df.groupby("raceId")["best_time"]
    .std()
    .reset_index()
    .rename(columns={"best_time": "grid_spread_time"})
)

grid_spread_time.describe()

#### Grid shake-up
Grid shake-up measures how far drivers qualified from their usual season performance. Bigger shake-up means the grid is more unusual. These out-of-position starts typically lead to more overtakes and more unpredictable race behavior, making it a subtle but meaningful predictor of excitement.

In [ ]:

# ---------- GRID SHAKE-UP ----------
# seasonal qualifying average per driver
qual_df = qual_df.sort_values(['year', 'round', 'driverId'])

season_avg = (
    qual_df.groupby(['year', 'driverId'])['position']
    .mean()
    .reset_index()
    .rename(columns={'position': 'season_avg_position'})
)


qual_pos = qual_df.merge(season_avg, on=["year", "driverId"], how="left")
qual_pos["shake"] = (qual_pos["position"] - qual_pos["season_avg_position"]).abs()
'''
qual_df['season_avg_pos'] = (
    qual_df.groupby(['year', 'driverId'])['position']
    .transform(lambda x: x.expanding().mean().shift(1))
)
qual_df['season_avg_pos'] = qual_df['season_avg_pos'].fillna(qual_df['position'])

# 4. Calculate the Shake
# How far is their current grid slot from their running average?
qual_df['shake'] = (qual_df['position'] - qual_df['season_avg_pos']).abs()
'''
grid_shakeup = (
    qual_pos.groupby("raceId")["shake"]
    .mean()
    .reset_index()
    .rename(columns={"shake": "grid_shakeup"})
)

grid_shakeup.describe()

#### Title Tension
Title tension summarizes how close the championship fight is. While it mainly captures fan hype rather than hard race dynamics, close title battles can increase driver aggression and strategic risk, making it a useful context feature even if the direct correlation with our label is small.

In [ ]:
# ---------- TITLE TENSION ----------
title_tension = (
    stand_df.groupby("raceId")
    .apply(lambda df: df.sort_values("points", ascending=False).head(3)["points"].std())
    .reset_index(name="title_tension")
)

title_tension.describe()

#### Altitude
Altitude is a track-level feature that affects engine power, cooling, and drag. Certain high-altitude tracks tend to produce more mechanical issues or unusual pace differences. This makes altitude a small but potentially relevant contextual factor, even though its direct correlation with excitement is mild.

In [ ]:
# ---------- ALTITUDE ----------
altitude = circuits_df[["circuitId", "alt"]].rename(columns={"alt": "altitude"})

Our pre-race features reflect different aspects of the race weekend: grid spread measures how mixed the starting order is; grid shake-up captures how unusual drivers’ qualifying results are relative to their season; title tension describes the competitive pressure in the championship; and altitude represents circuit-specific environmental difficulty. While none of these features individually guarantee an exciting race, each provides contextual cues that, when combined, contribute meaningful predictive signal.

In [ ]:
# ---------- MERGE EVERYTHING ----------
eda_df = (
    main_races
    .merge(labels_df, on="raceId", how="left")
    .merge(grid_spread_time, on="raceId", how="left")
    .merge(grid_shakeup, on="raceId", how="left")
    .merge(title_tension, on="raceId", how="left")
    .merge(altitude, on="circuitId", how="left")
)

eda_df.head()

In [ ]:
eda_df.isna().sum()

In [ ]:
eda_df.dropna(inplace=True)

In [ ]:
features = [
    #"grid_spread_time", 
    "grid_shakeup", "title_tension", "precipitation"
    #"altitude", "temperature", "precipitation"
]

#demonstrates correlation between data and how exciting the race is
eda_df[features + ["is_exciting"]].corr()["is_exciting"]

In [ ]:
plt.figure(figsize=(8,5))
sns.kdeplot(data=eda_df, x="grid_spread_time", hue="is_exciting", fill=True, common_norm=False)
plt.title("Distribution of Grid Spread for Exciting vs Non-Exciting Races")
plt.xlabel("Grid Spread (Std Dev of Qualifying Positions)")
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
sns.kdeplot(data=eda_df, x="grid_shakeup", hue="is_exciting", fill=True, common_norm=False)
plt.title("Distribution of Grid Shake-Up for Exciting vs Non-Exciting Races")
plt.xlabel("Grid Shake-Up (Mean Deviation from Driver’s Seasonal Average)")
plt.show()

#### Grid Shake-Up vs. Race Excitement
Grid shake-up measures how unusual the starting grid is compared to each driver’s typical qualifying performance that season. Higher shake-up values mean more drivers are starting far from where they “normally” qualify (e.g., a top driver starting P12, or a midfield driver unexpectedly lining up P5).

The KDE plot below compares the distribution of shake-up between exciting and non-exciting races:

- Both groups peak around 2.0–2.5, meaning most races involve some moderate amount of qualifying variability.

- The curve for exciting races is slightly shifted to the right, showing more density between 2.5–4.0.

- This suggests that exciting races tend to have a bit more qualifying chaos — cars starting out of position, which naturally leads to more overtaking and more dynamic race scenarios.

Even though the difference between the two distributions is subtle, it aligns with Formula 1 intuition: When the grid is mixed up, the race is usually more eventful. However, the heavy overlap also makes it clear that qualifying shake-up is not the sole source of excitement — many unpredictable factors on race day (weather, crashes, safety cars, tyre issues) can overshadow what happens on Saturday.

Exciting races show a slightly higher grid shake-up distribution, meaning more drivers start out of position compared to their seasonal average — a factor that tends to lead to more overtakes and race variability.

In [ ]:
# Scale train and test
def scaleDatasets(train_df, test_df):
    scaler = StandardScaler()

    # Fit only on training data
    scaler.fit(train_df)

    # Transform both
    train_scaled = scaler.transform(train_df)
    test_scaled = scaler.transform(test_df)

    return train_scaled, test_scaled


In [ ]:
#Drop features we don't need from the dataframe and split it into training and test sets
#also drop the is_exciting stat from the test datframe
def dfSplit(eda_df, features):
    feature_df = eda_df[features + ["is_exciting"]]
    size = len(feature_df)

    feature_df = feature_df.sample(frac = 1, random_state = 0)
    train_df = feature_df.iloc[:int(size * 0.8)]
    train_labels = train_df.pop("is_exciting")
    test_df = feature_df.iloc[int(size * 0.8):]
    test_labels = test_df.pop("is_exciting")

    return train_df, train_labels, test_df, test_labels

train_df, train_labels, test_df, test_labels = dfSplit(eda_df, features)
train_df, test_df = scaleDatasets(train_df, test_df)

### Baseline

In [ ]:
dummy = DummyClassifier(strategy="most_frequent")
dummy.fit(train_df, train_labels)
y_dummy = dummy.predict(test_df)

print("Dummy baseline report:")
print(classification_report(test_labels, y_dummy, zero_division=0))
print("Confusion matrix (dummy):\n", confusion_matrix(test_labels, y_dummy))

### Logistic Regression

In [ ]:
def train_logistic_model(X_train, y_train, c = 1):
    model = LogisticRegression(class_weight = "balanced", max_iter = 1000, C = c)
    model.fit(X_train, y_train)

    return model

logisticModel = train_logistic_model(train_df, train_labels)

In [ ]:
def test_logistic_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    #print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred, zero_division = 0))

test_logistic_model(logisticModel, test_df, test_labels)

In [ ]:
def ablation(features, ind):
    #for f in features:
    print(f"Removed feature: {features[ind]}")
    new_features = features.copy()
    #new_features.remove(f)
    new_features.pop(ind)
    train_df, train_labels, test_df, test_labels = dfSplit(eda_df, new_features)
    train_df, test_df = scaleDatasets(train_df, test_df)
    logisticModel = train_logistic_model(train_df, train_labels)
    test_logistic_model(logisticModel, test_df, test_labels)


In [ ]:
ablation(features, 0)

In [ ]:
ablation(features, 1)

In [ ]:
ablation(features, 2)

In [ ]:
ablation(features, 3)

### Random Forest

In [ ]:
def train_rf_model(X_train, y_train):
    model = RandomForestClassifier(n_estimators=100, class_weight="balanced", random_state=0)
    model.fit(X_train, y_train)

    return model

rfModel = train_rf_model(train_df, train_labels)
def test_rf_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    #print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))

test_rf_model(rfModel, test_df, test_labels)

### LinearSVC

In [ ]:
def train_svc(X_train, y_train):
    model = LinearSVC(class_weight='balanced', max_iter=10000)
    model.fit(X_train, y_train)
    return model

svcModel = train_svc(train_df, train_labels)


def test_svc(model, X_test, y_test):
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred, zero_division=0))

test_svc(svcModel, test_df, test_labels)